<a href="https://colab.research.google.com/github/etemadism/Courses/blob/main/00_Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Introduction**

This Colab notebook provides a comprehensive workflow for preprocessing and cleaning datasets in preparation for data analysis and machine learning tasks. The steps outlined are particularly useful when working with raw or structured datasets that may contain missing values, outliers, or categorical data requiring transformation.

The notebook utilizes a dataset originally stored in `.sav` format (SPSS file) and demonstrates how to efficiently load, clean, and prepare it for further analysis. The key preprocessing tasks include:

1. **Data Loading**: Reading `.sav` files and converting them to a format suitable for Python analysis.
2. **Label Encoding**: Transforming categorical variables into numerical representations.
3. **Handling Missing Values**: Imputing missing data and removing columns with excessive missing values.
4. **Outlier Removal**: Identifying and filtering out extreme values based on standard deviation thresholds (optional).
5. **Data Scaling**: Applying normalization or standardization to numeric features.
6. **One-Hot Encoding**: Encoding categorical variables into binary columns to enable machine learning models to process them effectively.

---
**Data Availability**

For learning purposes, you can access the dataset for this analysis on GitHub. The dataset is hosted on the GitHub account with the username **etemadism**. You can download the data directly from the repository to follow along with the notebook.

GitHub Repository: https://github.com/etemadism

**Author: Ali Etemadi**

Tehran University of Medical Sciences, Tehran, Iran
---

In [85]:
import pandas as pd
!pip install pyreadstat
###load .sav data
data_org=pd.read_spss("/content/preprocessing_sample.sav")
data_org.to_csv("/content/preprocessing_sample.csv")

In [86]:
data_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   ID                         400 non-null    float64 
 1   Age                        400 non-null    float64 
 2   sex                        400 non-null    category
 3   Height                     400 non-null    float64 
 4   Weight                     400 non-null    float64 
 5   Waistcm                    400 non-null    float64 
 6   Hipcm                      400 non-null    float64 
 7   Glucose                    399 non-null    float64 
 8   LDL_Chol                   399 non-null    float64 
 9   HDL_Chol                   399 non-null    float64 
 10  Cholestrol                 399 non-null    float64 
 11  Triglyceride               399 non-null    float64 
 12  mean_SBP                   400 non-null    float64 
 13  mean_DBP                   400 non-

In [90]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder


# Convert categorical features to numerical using Label Encoding
def label_encode(df):
    for column in df.select_dtypes(include=['object', 'category']).columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    return df

# Apply Label Encoding to the dataset
data = label_encode(data)
# Handling Missing Values
def handle_missing_values(df, missing_threshold=0.2):
    """
    Handles missing values in the dataset.
    - Columns with missing percentage below the threshold are imputed.
    - Columns with missing percentage above the threshold are dropped.

    Parameters:
    df (pd.DataFrame): Input dataset.
    missing_threshold (float): Threshold for removing columns (default is 20%).

    Returns:
    pd.DataFrame: Cleaned dataset with handled missing values.
    """
    missing_percentage = df.isnull().mean()  # Calculate missing percentages for each column
    columns_to_drop = missing_percentage[missing_percentage > missing_threshold].index  # Identify columns to drop

    print(f"Number of columns dropped due to missing values: {len(columns_to_drop)}")
    print(f"Columns dropped: {list(columns_to_drop)}")

    df = df.drop(columns=columns_to_drop)  # Drop columns with high missing percentage

    # Impute remaining missing values
    imputer = SimpleImputer(strategy='mean')  # Use mean for imputation
    df[df.columns] = imputer.fit_transform(df)

    return df


# Handling Outliers: Remove outliers more than 3 standard deviations
def remove_outliers(df, threshold=3):
    """
    Removes outliers from numeric columns using the specified threshold (default is 3 standard deviations).

    Parameters:
    df (pd.DataFrame): Input dataset.
    threshold (int): Threshold in terms of standard deviations for detecting outliers.

    Returns:
    pd.DataFrame: Dataset with outliers removed.
    """
    numeric_columns = df.select_dtypes(include=[np.number]).columns

    for column in numeric_columns:
        mean = df[column].mean()
        std_dev = df[column].std()
        # Keep only rows within the threshold
        df = df[df[column].between(mean - threshold * std_dev, mean + threshold * std_dev)]

    return df

# Data Normalization/Standardization
def normalize_standardize(df, method='standardize'):
    """
    Normalizes or standardizes numeric columns in the dataset.
    - Standardize: Z-score normalization.
    - Normalize: Min-Max scaling.

    Parameters:
    df (pd.DataFrame): Input dataset.
    method (str): 'standardize' or 'normalize'.

    Returns:
    pd.DataFrame: Transformed dataset.
    """
    numeric_columns = df.select_dtypes(include=[np.number]).columns

    if method == 'standardize':  # Z-score normalization
        scaler = StandardScaler()
    elif method == 'normalize':  # Min-Max scaling
        scaler = MinMaxScaler()
    else:
        raise ValueError("Invalid method. Choose 'standardize' or 'normalize'.")

    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    return df

# One-Hot Encoding for Categorical Variables
def one_hot_encode(df):
    """
    Performs one-hot encoding on categorical variables.

    Parameters:
    df (pd.DataFrame): Input dataset.

    Returns:
    pd.DataFrame: Dataset with categorical variables one-hot encoded.
    """
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)  # Avoid multicollinearity
    return df

# Example: Applying the functions
# Assuming `data` is your original dataset
data= data.iloc[:,1:84]
data= label_encode(data)
data = handle_missing_values(data)          # Handle missing values
#data = remove_outliers(data)                # Remove outliers
data = normalize_standardize(data, method='standardize')  # Standardize data
data = one_hot_encode(data)                 # Encode categorical variables

# Display the final cleaned dataset
data


Number of columns dropped due to missing values: 0
Columns dropped: []


,Age,sex,Height,Weight,Waistcm,Hipcm,Glucose,LDL_Chol,HDL_Chol,Cholestrol,...,Vitamin_B3,Folate,Vitamin_B5,Vitamin_C,Vitamin_K,Vitamin_B1,Vitamin_B2,Vitamin_B6,Vitamin_B12,Biotin
0,-1.142905,-0.932287,-0.147903,0.777140,1.426278,1.048799,-0.577478,0.422044,-0.756534,0.960045,...,0.383887,-0.339752,0.268376,-0.076403,-0.350691,-0.235485,-0.232692,-0.448631,-0.463583,0.137990
1,0.586387,-0.932287,-1.189666,-0.425233,1.156447,0.629227,1.668591,-0.922049,0.704817,-0.429958,...,-0.370554,-0.755077,-0.448243,-0.372665,-0.281971,-0.966524,-0.513630,-0.394817,-0.340440,-0.601349
2,0.271971,1.072631,0.619712,0.932285,0.257013,-0.524596,-0.084439,-1.196353,-0.272351,-0.721410,...,5.223749,1.583431,2.039217,5.365363,-0.409164,1.396527,1.627920,3.304221,-0.130583,0.389110
3,2.472888,-0.932287,-0.805859,-1.681906,-1.631798,-1.153954,0.819467,-0.400870,-0.272351,-0.362700,...,-0.236321,-0.359018,0.298556,-0.225847,-0.378166,-0.772543,0.351659,-0.201345,-0.071736,-0.247362
4,-0.671280,1.072631,1.277668,-0.386446,-0.597449,-0.839275,-0.933562,0.230031,-0.404401,-0.205764,...,2.805061,3.254777,1.632127,0.597636,5.212636,1.598525,1.963440,1.974258,0.508797,2.154004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,-0.985697,1.072631,0.564882,-0.037371,0.122098,-0.314810,-0.604869,0.943223,-0.633287,0.691012,...,1.106013,-0.340440,0.265568,-0.698454,-0.382649,0.668695,0.245705,-0.467850,-0.285018,-0.104052
396,1.529638,-0.932287,-1.299325,-0.580378,-0.102760,0.314548,-0.413132,1.135236,1.690788,1.161820,...,2.152537,-0.535855,0.321016,0.041613,-0.406794,-0.306024,-0.507208,0.494387,-0.345266,-0.331471
397,-0.985697,1.072631,0.510053,-0.774309,-1.721742,-1.258847,-0.467914,-0.812327,-0.201925,-0.990443,...,2.909465,1.111408,2.152921,1.944446,0.204323,2.236581,1.478622,0.818549,0.555345,0.879246
398,-0.985697,1.072631,0.948690,-1.394888,-1.811685,-1.468633,-0.248785,-1.498088,0.185421,-1.416412,...,-0.087173,-0.483561,-0.090987,-0.322760,-0.518072,-0.049519,-0.699852,-0.338441,-0.028456,-0.074805


In [88]:
# Display the encoded data
data[['sex', 'Cancer01', 'HighFBS']]


,sex,Cancer01,HighFBS
0,0,1,0
1,0,1,1
2,1,1,1
3,0,0,1
4,1,1,0
...,...,...,...
395,1,1,0
396,0,1,0
397,1,1,0
398,1,1,0


In [89]:
data_org[['sex', 'Cancer01', 'HighFBS']]

,sex,Cancer01,HighFBS
0,Female,No,NO
1,Female,No,YES
2,Male,No,YES
3,Female,,YES
4,Male,No,NO
...,...,...,...
395,Male,No,NO
396,Female,No,NO
397,Male,No,NO
398,Male,No,NO


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 83 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        400 non-null    float64
 1   sex                        400 non-null    float64
 2   Height                     400 non-null    float64
 3   Weight                     400 non-null    float64
 4   Waistcm                    400 non-null    float64
 5   Hipcm                      400 non-null    float64
 6   Glucose                    400 non-null    float64
 7   LDL_Chol                   400 non-null    float64
 8   HDL_Chol                   400 non-null    float64
 9   Cholestrol                 400 non-null    float64
 10  Triglyceride               400 non-null    float64
 11  mean_SBP                   400 non-null    float64
 12  mean_DBP                   400 non-null    float64
 13  CK                         400 non-null    float64